# GCCompany Demo: 대화형 검색


In [ ]:
!pip install langchain
!pip install langchain_community
!pip install boto3
!pip install opensearch-py

In [4]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import boto3
import json
import textwrap

# 컨텍스트 없이 물어보기


In [5]:
query_text = "겨울에 가족과 함께 가기 좋은 숙소"

In [18]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
}

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
)  # Claude 속성 구성

messages = [HumanMessage(content=query_text)]

print(textwrap.fill(llm(messages).content, 80))

겨울에 가족과 함께 즐길 수 있는 숙소로는 다음과 같은 곳들이 좋습니다.  1. 스키리조트 콘도/펜션 - 스키를 즐기며 가족과 함께 겨울
액티비티를 만끽할 수 있습니다. - 대표적인 곳으로는 휘닉스파크, 하이원리조트, 비발디파크 등이 있습니다.  2. 온천/스파 리조트 - 따듯한
온천수에 몸을 담그며 겨울철 추위를 녹일 수 있습니다. - 일상에서 벗어나 휴식을 취할 수 있는 곳입니다. - 대표적인 곳으로는 해운대 온천,
아산 스파비스 등이 있습니다.  3. 전원주택/펜션 - 도심을 벗어나 한적한 시골 분위기를 만끽할 수 있습니다.  - 벽난로가 있어 가족들과
둘러앉아 보내기에 좋습니다. - 대표적인 곳으로는 강원도, 경기 가평, 여주 등지에 많이 있습니다.  4. 키즈 호텔/리조트 - 아이들을 위한
실내 놀이시설과 프로그램이 마련되어 있습니다. - 대표적인 곳으로는 롯데리조트, 코리아나 호텔 등이 있습니다.  가족 구성원 수와 예산, 취향
등을 고려하여 알맞은 숙소를 선택하면 좋은 겨울 가족여행이 될 것입니다.


## RAG을 통한 대화형 검색


### LangChain 구성하기


In [7]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region="us-east-1", service="es")

aos_host = "search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws"
opensearch_auth = ("admin", "Make1StepForward!")

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

In [8]:
import numpy as np
from langchain_community.embeddings import BedrockEmbeddings
from requests_aws4auth import AWS4Auth
from typing import Any, Dict, Iterable, List, Optional, Tuple, Callable
from typing import Type

aos_url = "https://" + aos_host

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

# this is just an example, you would need to change these values to point to another opensearch instance
docsearch = OpenSearchVectorSearch(
    index_name="hotels_index",
    embedding_function=embeddings,
    opensearch_url=aos_url,
    http_auth=auth,
    connection_class=RequestsHttpConnection,
)


class SimiliarOpenSearchVectorSearch(OpenSearchVectorSearch):

    def relevance_score(self, distance: float) -> float:
        return distance

    def _select_relevance_score_fn(self) -> Callable[[float], float]:
        return self.relevance_score


open_search_vector_store = SimiliarOpenSearchVectorSearch(
    index_name="hotels_index",
    embedding_function=embeddings,
    opensearch_url=aos_url,
    http_auth=auth,
    connection_class=RequestsHttpConnection,
)

In [19]:
# you can specify custom field names to match the fields you're using to store your embedding, document text value, and metadata
docs = open_search_vector_store.similarity_search_with_score(
    query_text,
    search_type="approximate_search",
    space_type="cosinesimil",
    k=10,
)

# docs_ = open_search_vector_store.similarity_search_with_score(question, k=5)

print("found document number:" + str(len(docs)))

print("opensearch results:\n")
for doc in docs:
    print(doc)
    print("\n-----------------")

found document number:10
opensearch results:

(Document(page_content='평창 그림 같은 하우스 펜션은 평창군에 위치해 있으며, 바론 밸리와 허브나라농원에서 15분 거리에 있습니다. 오대산국립공원과 월정사 편백숲에서도 가깝습니다. 주변 관광지로는 바론 밸리, 허브나라농원, 오대산국립공원, 월정사 편백숲, 대관령 스카이랜치, 대관령 삼양랜치가 있습니다.', metadata={'address': '강원도 평창군 봉평면 경강로 464-18', 'city': '평창군', 'vector_field': [-0.15820312, -0.03564453, 0.03930664, 0.66015625, 0.1875, -0.24316406, 0.36328125, 0.00012397766, -0.29101562, -0.40820312, 0.67578125, -0.17382812, -0.24902344, 0.43554688, 0.45703125, -0.008972168, 0.17382812, 0.13671875, -0.3515625, 0.140625, -0.0020599365, -0.40234375, 0.033935547, 0.08544922, 0.064941406, 0.15039062, 0.13183594, 0.58984375, 0.2890625, -0.72265625, -0.40820312, 0.22753906, 0.26171875, -0.390625, -0.014404297, -0.203125, 0.09375, -0.32226562, -0.125, 0.088378906, -0.87890625, 0.16699219, 0.51953125, -0.119628906, 0.06933594, 0.23925781, -0.21875, 0.67578125, 0.111816406, -0.28515625, 0.33984375, 0.26367188, -0.10644531, -0.515625, -0.17480469, -0.19824219, -0.0761

In [10]:
retriever = open_search_vector_store.as_retriever(
    # search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "lambda_mult": 0.25},
)

In [11]:
query = query_text
relevant_documents = retriever.get_relevant_documents(query)

for doc in relevant_documents:
    print(doc.page_content)

평창 그림 같은 하우스 펜션은 평창군에 위치해 있으며, 바론 밸리와 허브나라농원에서 15분 거리에 있습니다. 오대산국립공원과 월정사 편백숲에서도 가깝습니다. 주변 관광지로는 바론 밸리, 허브나라농원, 오대산국립공원, 월정사 편백숲, 대관령 스카이랜치, 대관령 삼양랜치가 있습니다.
평창군에 위치한 평창 그림 같은 하우스 펜션은 2성급 펜션입니다. 주요 관광지로는 바론 밸리, 허브나라, 오대산국립공원, 월정사 편백숲, 대관령 스카이랜치, 대관령 삼양랜치가 있습니다. 주소는 강원도 평창군 봉평면 경강로 464-18입니다. 위치는 37.59699|128.31194입니다.
영월 우리들펜션은 영월군 무릉도원면에 위치해 있으며, 서한광광농원과 가까운 거리에 있습니다. 요선정 정자와 요선암바위, 상원사 사찰 등의 주요 관광지가 인근에 있습니다. 객실에서 편안한 휴식을 취할 수 있습니다. 주변에는 치악산국립공원, 세명대학교, 구룡사 사찰, 주천강 강변 휴양림 등의 관광지가 있으며, 원주공항이 가장 가까운 공항입니다. 위치는 경도 128.19226, 위도 37.31197입니다.
평창 친절한 가쁜씨 의 펜션은 피닉스 파크 스키 리조트 근처에 위치해 있으며, 바롱 밸리와 허브나라 농원에서 가까운 거리에 있습니다. 주요 관광지로는 월정사 편백숲, 대관령 스카이랜치, 대관령 삼양랜치가 있습니다. 5개의 객실이 있으며, 체크인 시 정부에서 발급한 신분증과 신용카드 또는 현금 보증금이 필요합니다. 특별 요청 사항은 체크인 시 가능 여부에 따라 추가 요금이 부과될 수 있습니다.
평창군 진부면에 위치한 휴가 펜션은 오대산국립공원과 한국식물원에서 15분 이내 거리에 있습니다. 월정사 편백 숲과 월정사에서도 가깝습니다. 주변에는 알펜시아 스키 리조트, 용평 워터파크, 용평 골프 클럽 등의 관광지가 있으며 양양국제공항이 가장 가까운 공항입니다. 펜션에는 주방, 전자레인지, 전기밥솥 등의 시설이 갖춰져 있습니다.
평창솔숲펜션은 평창군에 위치한 2성급 펜션입니다. 주소는 강원도 평창군 평창읍 용평리 

In [12]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=10, return_messages=True)

In [13]:
from langchain import PromptTemplate

template = """

You are a korean tour guide who are experts in Korean hotels. Please answer the <QUESTION>.
Always say the hotel name, not 'this hotel' or 'that hotel'.

<CONTEXT>
{context}
</CONTEXT>

<QUESTION>
{question}
</QUESTION>
"""

prompt_template = PromptTemplate(input_variables=["question", "context"], template=template)

In [14]:
condense_template = """
Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>
{chat_history}
</chat-history>

<follow-up-question>
{question}
</follow-up-question>

standalone question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

In [20]:
from langchain.chains import ConversationalRetrievalChain

memory.clear()

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
)

chat_response = conversation_with_retrieval.invoke({"question": query_text})

print(textwrap.fill(chat_response["answer"], 80))

겨울에 가족과 함께 가기 좋은 숙소로 평창 그림 같은 하우스 펜션을 추천합니다. 평창 그림 같은 하우스 펜션은 바론 밸리와 허브나라농원에서
15분 거리에 있어 겨울 눈꽃 관광을 즐기기에 좋습니다. 또한 오대산국립공원과 월정사 편백숲에서도 가까워 자연 경관을 만끽할 수 있습니다. 가족
단위 여행객을 위한 편의시설을 갖추고 있어 편안한 휴식을 취할 수 있을 것입니다.


In [21]:
chat_response = conversation_with_retrieval.invoke(
    {"question": "평창 그림 같은 하우스에 주방이 있어?"}
)

print(textwrap.fill(chat_response["answer"], 80))

평창 그림 같은 하우스 펜션은 바론 밸리, 허브나라 농원, 오대산국립공원, 월정사 편백숲 등 주요 관광지와 가까워 겨울 가족 여행지로
적합합니다. 펜션에는 주방 시설이 갖추어져 있어 가족들이 직접 요리를 할 수 있습니다. 또한 객실마다 개인 테라스가 있어 가족들이 함께 여유로운
시간을 보낼 수 있습니다. 무료 WiFi와 평면 TV도 제공되므로 인터넷 사용과 영화 시청도 가능합니다. 따라서 평창 그림 같은 하우스 펜션은
주방 시설과 다양한 편의시설을 갖추고 있어 가족 단위 겨울 여행객들에게 안성맞춤입니다.


In [22]:
chat_response = conversation_with_retrieval.invoke({"question": "주변 관광지는 어디야?"})

print(textwrap.fill(chat_response["answer"], 80))

평창 그림 같은 하우스 펜션 주변에는 바론 밸리, 허브나라농원, 오대산국립공원, 월정사 편백숲, 대관령 스카이랜치, 대관령 삼양랜치 등의
관광지가 있습니다. 특히 바론 밸리와 허브나라농원은 펜션에서 15분 거리에 있어 가까이에서 즐길 수 있습니다. 오대산국립공원과 월정사 편백숲도
인접해 있어 자연을 만끽하기에 좋은 위치입니다.
